In [36]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Polygon, LineString, MultiLineString, Point, GeometryCollection
pd.options.display.max_columns = 100
import pygeos

In [37]:
#Assigning Path
path_road = r'Z:\Masterfile\1. Shapefile\5. Road\1. All Road'
path_poi = r'Z:\Masterfile\1. Shapefile\4. POI'
path_zmp = r'Z:\000. Processing\REGION I\ZMP'

In [38]:
#Preparing and Reading Files
poi = gpd.read_file(os.path.join(path_poi, 'poi_priority.gpkg'))
road = gpd.read_file(os.path.join(path_road, 'Road_With_Characteristics.gpkg'))
zmp = gpd.read_file(os.path.join(path_zmp, 'ID_ZMP_MASTERFILE_REGION_I.shp'))

In [39]:
#Changing Prjection and creating road length
zmp = zmp.to_crs(epsg = 4326)
# road = road.to_crs(epsg = 32748)
# road['length'] = road.length
# road = road.to_crs(epsg = 4326)

In [40]:
# #Changing Projection
# zmp = zmp.to_crs(epsg = 4326)
# #Join zmp with road
# zmp_road = gpd.sjoin(road, zmp, how = 'right', predicate = 'intersects')
# #Creating rank
# zmp_road['rank'] = zmp_road.groupby(['ZMP_CODE'])['length'].rank(ascending = False)
# #Applying ZMP Category
# def zmp_cat(a):
#     if a=='>5 m':
#         return 'ZMP Commercial'
#     else:
#         return 'ZMP Non Commercial'
# zmp_road['zmp_category'] = zmp_road['Perkiraan Luas'].apply(lambda x: zmp_cat(x))
# zmp_road = zmp_road[['ZMP_CODE','zmp_category']]

# ZMP With POI Class

In [41]:
#Merging ZMP with POI
zmp_join = gpd.sjoin(poi, zmp, how = 'right', predicate='intersects')

In [42]:
#Filtering not joined and with class
zmp_poi = zmp_join[~zmp_join['POI_Class'].isnull()]
zmp_no_poi = zmp_join[zmp_join['POI_Class'].isnull()][['ZMP_CODE','ZMP_NAME']]
zmp_no = zmp.merge(zmp_no_poi, how = 'inner', on ='ZMP_CODE').rename(columns= {'ZMP_NAME_x':'ZMP_NAME'}).drop(columns = 'ZMP_NAME_y')

In [43]:
#Join Nearest no poi zmp with poi
zmp_no.to_crs(epsg = 32748, inplace = True)
poi.to_crs(epsg = 32748, inplace = True)
zmp_no_join = gpd.sjoin_nearest(poi, zmp_no, how = 'right')

In [44]:
# Create ZMP Summary with POI Class
zmp_poi_dissolve = zmp_poi.pivot_table(index = ['ZMP_CODE','ZMP_NAME', 'KECA', 'CITY_right'], values = 'Sub_Class', columns='POI_Class', aggfunc='count').reset_index().fillna(0)
zmp_no_poi_dissolve = zmp_no_join.pivot_table(index = ['ZMP_CODE','ZMP_NAME', 'KECA', 'CITY_right'], values = 'Sub_Class', columns='POI_Class', aggfunc='count').reset_index().fillna(0)
zmp_full = pd.concat([zmp_poi_dissolve, zmp_no_poi_dissolve])

In [45]:
#Define zmp category function
def category(a):
    if a['Religion'] >=1:
        return 'Residential or Religion'
    elif a['School']>=1:
        return 'Education'
    elif a['CVS']>=1:
        return 'Commercial'
    elif a['Gov']>=1:
        return 'Goverment'
    elif a['Health Center']>=1:
        return 'Health Center'
    elif a['Transportation']>=1:
        return 'Transportation'
    else:
        return 'Commercial'

In [46]:
zmp_full['zmp_category'] = zmp_full[['CVS', 'Gov', 'Health Center', 'Religion', 'School',
       'Transportation']].apply(lambda x: category(x), axis = 1)

In [47]:
zmp_full.head()

POI_Class,ZMP_CODE,ZMP_NAME,KECA,CITY_right,CVS,Gov,Health Center,Religion,School,Transportation,zmp_category
0,3171010_005,ISTN Jagakarsa,Jagakarsa,Kota Jakarta Selatan,1.0,0.0,1.0,0.0,1.0,0.0,Education
1,3171010_008,RSU Andhika,Jagakarsa,Kota Jakarta Selatan,0.0,1.0,0.0,1.0,0.0,0.0,Residential or Religion
2,3171010_010,Kantor Kelurahan Cipedak,Jagakarsa,Kota Jakarta Selatan,1.0,0.0,0.0,0.0,0.0,0.0,Commercial
3,3171010_012,Jl. Sirsak Moch. Kahfi II,Jagakarsa,Kota Jakarta Selatan,0.0,0.0,1.0,0.0,0.0,0.0,Health Center
4,3171010_017,Masjid Daarul Iman Ciganjur,Jagakarsa,Kota Jakarta Selatan,1.0,0.0,0.0,0.0,0.0,0.0,Commercial


In [48]:
#Finalizing ZMP
zmp_final = zmp.merge(zmp_full, how ='left', on = 'ZMP_CODE').rename(columns = {'ZMP_NAME_x':'ZMP_NAME'})[['PROVINCE', 'KECAMATAN', 'CITY', 'ZMP_CODE', 'ZMP_NAME','zmp_category','geometry']]
zmp_final.to_file(r'Z:\Masterfile\1. Shapefile\8. ZMP\zmp_masterfile.gpkg')

## Uploading to Google Sheets

In [49]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
#Authorize the API
scope = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file'
    ]
file_name = r'C:\Users\User\OneDrive - Migo (1)\Documents\Migo\Data Project\Masterfile\Credentials\update-mds-d3d8ff0bb675.json'
creds = ServiceAccountCredentials.from_json_keyfile_name(file_name,scope)
client = gspread.authorize(creds)

spreadsheetId = "1P3mo9DnaZ9Kv3slU7AWSQCBuOJZgKPUMA5WAMhHXypY"
sheetId = "156203018"

sh = client.open_by_key(spreadsheetId)
worksheet = sh.get_worksheet(4)
worksheet

<Worksheet 'zmp_category' id:156203018>

In [50]:
worksheet.update([zmp_full.columns.values.tolist()] + zmp_full.values.tolist())

{'spreadsheetId': '1P3mo9DnaZ9Kv3slU7AWSQCBuOJZgKPUMA5WAMhHXypY',
 'updatedRange': 'zmp_category!A1:K8959',
 'updatedRows': 8959,
 'updatedColumns': 11,
 'updatedCells': 98549}